In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm
import dateutil
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")

In [3]:
cutoff_start_year = '2014'
cutoff_start_date = datetime.strptime(cutoff_start_year+'-01-01','%Y-%m-%d')

In [4]:
match_summary_df = pd.read_csv('csv_data/match_list.csv',parse_dates=['date'],date_parser=custom_date_parser)
recent_match_summary_df = match_summary_df[match_summary_df['date']>=cutoff_start_date]
recent_match_summary_df.shape

(680, 11)

In [5]:
country_set = set(recent_match_summary_df['first_innings'].unique()).union(set(recent_match_summary_df['second_innings'].unique()))
country_list = list(country_set)


In [6]:
country_list

['South Africa',
 'Namibia',
 'New Zealand',
 'Oman',
 'United States of America',
 'Bangladesh',
 'Nepal',
 'Canada',
 'India',
 'Australia',
 'Scotland',
 'Ireland',
 'Hong Kong',
 'Zimbabwe',
 'Pakistan',
 'Kenya',
 'England',
 'West Indies',
 'Papua New Guinea',
 'Sri Lanka',
 'Netherlands',
 'United Arab Emirates',
 'Afghanistan']

In [7]:
location_list = list(set(recent_match_summary_df['location'].unique()))
location_list

['Leeds',
 'Bulawayo Athletic Club',
 'Abu Dhabi',
 'Sharjah Cricket Stadium',
 'Sharjah',
 'Dhaka',
 'Paarl',
 'Melbourne Cricket Ground',
 'Lincoln',
 'Bulawayo',
 'Delhi',
 'Mumbai',
 'Dubai International Cricket Stadium',
 'Nagpur',
 'Kolkata',
 'Melbourne',
 'Potchefstroom',
 'Harare Sports Club',
 'Colombo',
 'Guwahati',
 'Bridgetown',
 'Dublin',
 'Guyana',
 'Chennai',
 'Dunedin',
 'Sydney',
 'Harare',
 'Sydney Cricket Ground',
 'Pallekele International Cricket Stadium',
 'East London',
 'Rajkot',
 'Perth',
 'Bloemfontein',
 'Hyderabad',
 'Whangarei',
 'Taunton',
 'Kanpur',
 'St Lucia',
 'Kandy',
 'Aberdeen',
 'Kirtipur',
 'Hobart',
 'Indore',
 'Nelson',
 'Christchurch',
 'Fatullah',
 'Chattogram',
 'Port Elizabeth',
 'Chittagong',
 'Cape Town',
 'Napier',
 'Chester-le-Street',
 'Birmingham',
 'Belfast',
 'Deventer',
 'Galle International Stadium',
 'Brisbane',
 'London',
 'Mount Maunganui',
 'Pune',
 'Kuala Lumpur',
 'Johannesburg',
 'Lauderhill',
 'Bready',
 'Benoni',
 'Kochi',

In [8]:
team_location_list = country_list+location_list
#team_location_list

In [9]:
enc_map={}
decode_map={}

for count,team_loc in enumerate(team_location_list):
    vec = []
    for vec_val in range(len(team_location_list)):
        if count != vec_val:
            vec.append(0)
        else:
            vec.append(1)
    enc_map[team_loc]=vec
    decode_map[str(vec)]=team_loc
    

In [10]:
# country_enc_map={}
# enc_country_map={}

# for count,country in enumerate(country_list):
#     country_vec = []
#     for vec_val in range(len(country_list)):
#         if count != vec_val:
#             country_vec.append(0)
#         else:
#             country_vec.append(1)
#     country_enc_map[country]=country_vec
#     enc_country_map[str(country_vec)]=country
    

In [11]:
# location_list = list(set(recent_match_summary_df['location'].unique()))
# location_list

In [12]:
# loc_enc_map={}
# enc_loc_map={}

# for count,loc in enumerate(location_list):
#     loc_vec = []
#     for vec_val in range(len(location_list)):
#         if count != vec_val:
#             loc_vec.append(0)
#         else:
#             loc_vec.append(1)
#     loc_enc_map[loc]=loc_vec
#     enc_loc_map[str(loc_vec)]=loc

In [13]:
match_id_list = list(recent_match_summary_df['match_id'])

In [14]:
team_vector_list_train = []
opponent_vector_list_train = []
location_vector_list_train = []
first_innings_score_list_train = []
total_score_list_train =[]

team_vector_list_test = []
opponent_vector_list_test = []
location_vector_list_test = []
first_innings_score_list_test = []
total_score_list_test =[]

for selected_match_id in tqdm(match_id_list):
    is_train = recent_match_summary_df[recent_match_summary_df['match_id']==selected_match_id]['train_data'].values[0]
    if is_train:
        match_detail_df = pd.read_csv('csv_data/train/'+str(selected_match_id)+'.csv')
    else:
        match_detail_df = pd.read_csv('csv_data/test/'+str(selected_match_id)+'.csv')
        
    team = recent_match_summary_df[recent_match_summary_df['match_id']==selected_match_id]['first_innings'].values[0]
    opponent = recent_match_summary_df[recent_match_summary_df['match_id']==selected_match_id]['second_innings'].values[0]
    location = recent_match_summary_df[recent_match_summary_df['match_id']==selected_match_id]['location'].values[0]
    
    first_innings_score = match_detail_df[match_detail_df['team']==team]['total'].sum()
    total_runs = match_detail_df['total'].sum()
    
    team_vector = np.array(enc_map[team])
    opponent_vector = np.array(enc_map[opponent])
    location_vector = np.array(enc_map[location])
    
    if is_train:
        team_vector_list_train.append(team_vector)
        opponent_vector_list_train.append(opponent_vector)
        location_vector_list_train.append(location_vector)
        first_innings_score_list_train.append(first_innings_score)
        total_score_list_train.append(total_runs)
    else:
        team_vector_list_test.append(team_vector)
        opponent_vector_list_test.append(opponent_vector)
        location_vector_list_test.append(location_vector)
        first_innings_score_list_test.append(first_innings_score)
        total_score_list_test.append(total_runs)
        
    

In [15]:
#match_detail_df

In [16]:
recent_match_summary_df.columns

Index(['match_id', 'date', 'location', 'first_innings', 'second_innings',
       'winner', 'win_by', 'win_dif', 'toss_winner', 'player_of_match',
       'train_data'],
      dtype='object')

In [17]:
#np.array(team_vector_list_train).shape

In [18]:
team_vector_train = np.stack(team_vector_list_train)
opponent_vector_train = np.stack(opponent_vector_list_train)
location_vector_train = np.stack(location_vector_list_train)
Y_score_train = np.array(first_innings_score_list_train)
Y_total_train = np.array(total_score_list_train)

team_vector_test = np.stack(team_vector_list_test)
opponent_vector_test = np.stack(opponent_vector_list_test)
location_vector_test = np.stack(location_vector_list_test)
Y_score_test = np.array(first_innings_score_list_test)
Y_total_test = np.array(total_score_list_test)

In [19]:
team_vector_train.shape,opponent_vector_train.shape,location_vector_train.shape

((515, 137), (515, 137), (515, 137))

In [20]:
import keras as k
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.regularizers import l2

from keras.optimizers import Adam, Adadelta

In [56]:
def create_embedding_model(team_vector_len,opponent_vector_len,location_vector_len):
    team_input = Input((team_vector_len,),name="team_input")
    opponent_input = Input((opponent_vector_len,),name="opponent_input")
    location_input = Input((location_vector_len,),name="location_input")

    #team_output = Dropout(0.2)(team_input)
    team_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.1),name="team_1")(team_input)
    team_output = Dropout(0.2)(team_output)
    
    #opponent_output = Dropout(0.2)(opponent_input)
    opponent_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.1),name="opp_1")(opponent_input)
    opponent_output = Dropout(0.2)(opponent_output)
    
    #location_output = Dropout(0.2)(location_input)
    location_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.1),name="loc_1")(location_input)
    location_output = Dropout(0.2)(location_output)
    
    concat_out = Concatenate()([team_output, opponent_output,location_output])
    runs_output = Dropout(0.2)(concat_out)
    runs_output = Dense(1,name="final_score",use_bias=True, kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01),kernel_initializer='normal')(concat_out)
    
    
    team_model = Model(inputs=team_input,outputs=team_output)
    opponent_model = Model(inputs=opponent_input,outputs=opponent_output)
    location_model = Model(inputs=location_input,outputs=location_output)
    group_encode_model = Model(inputs=[team_input,opponent_input,location_input],
                               outputs=concat_out)
    
    runs_model = Model(inputs=[team_input,opponent_input,location_input],
                      outputs=runs_output)
    
    
    return team_model,opponent_model,location_model,group_encode_model,runs_model

In [57]:
# def create_embedding_model(team_vector_len,location_vector_len):
#     team_input = Input((team_vector_len,),name="team_input")
#     #opponent_input = Input((opponent_vector_len,),name="opponent_input")
#     location_input = Input((location_vector_len,),name="location_input")

#     team_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',kernel_regularizer=l2(0.01),name="team_1")(team_input)
    
#     #opponent_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',name="opp_1")(opponent_input)
    
#     location_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',kernel_regularizer=l2(0.01),name="loc_1")(location_input)
    
#     runs_output = Concatenate()([team_output,location_output])
#     runs_output = Dense(1,name="final_score",use_bias=True, kernel_initializer='normal',kernel_regularizer=l2(0.01))(runs_output)
    
    
#     team_model = Model(inputs=team_input,outputs=team_output)
#     #opponent_model = Model(inputs=opponent_input,outputs=opponent_output)
#     location_model = Model(inputs=location_input,outputs=location_output)
    
#     runs_model = Model(inputs=[team_input,location_input],
#                       outputs=runs_output)
    
    
#     return team_model,location_model,runs_model

In [58]:
team_model,opponent_model,location_model,group_encode_model,runs_model = create_embedding_model(team_vector_train.shape[1],
                                                                             opponent_vector_train.shape[1],
                                                                             location_vector_train.shape[1])

# team_model,location_model,runs_model = create_embedding_model(team_vector_train.shape[1],
#                                                                              location_vector_train.shape[1])


In [59]:
#group_encode_model.summary()

In [60]:
runs_model.compile(loss="mean_squared_error", metrics=["mean_absolute_percentage_error","mean_absolute_error"],optimizer=Adam(0.001))

In [61]:
runs_model.fit([team_vector_train,opponent_vector_train,location_vector_train], Y_score_train,
               validation_data=([team_vector_test,opponent_vector_test,location_vector_test],Y_score_test),epochs=120, batch_size=10)
#runs_model.fit([team_vector_train,location_vector_train], Y_score_train, epochs=1000, batch_size=10)


Epoch 1/120
52/52 [==============================] - 0s 7ms/step - loss: 72099.2578 - mean_absolute_percentage_error: 99.9682 - mean_absolute_error: 259.9359 - val_loss: 73324.4297 - val_mean_absolute_percentage_error: 99.9244 - val_mean_absolute_error: 262.0883
Epoch 2/120
52/52 [==============================] - 0s 4ms/step - loss: 71925.8750 - mean_absolute_percentage_error: 99.8305 - mean_absolute_error: 259.6044 - val_loss: 73060.6562 - val_mean_absolute_percentage_error: 99.7150 - val_mean_absolute_error: 261.5891
Epoch 3/120
52/52 [==============================] - 0s 3ms/step - loss: 71525.9141 - mean_absolute_percentage_error: 99.5024 - mean_absolute_error: 258.8286 - val_loss: 72545.1719 - val_mean_absolute_percentage_error: 99.3057 - val_mean_absolute_error: 260.6113
Epoch 4/120
52/52 [==============================] - 0s 3ms/step - loss: 70844.0156 - mean_absolute_percentage_error: 98.9435 - mean_absolute_error: 257.5043 - val_loss: 71734.0000 - val_mean_absolute_percentage

52/52 [==============================] - 0s 5ms/step - loss: 4553.5649 - mean_absolute_percentage_error: 24.0650 - mean_absolute_error: 52.6162 - val_loss: 4216.8169 - val_mean_absolute_percentage_error: 23.7916 - val_mean_absolute_error: 50.9986
Epoch 64/120
52/52 [==============================] - 0s 4ms/step - loss: 4501.7974 - mean_absolute_percentage_error: 24.2705 - mean_absolute_error: 51.9382 - val_loss: 4213.6572 - val_mean_absolute_percentage_error: 23.7876 - val_mean_absolute_error: 50.9631
Epoch 65/120
52/52 [==============================] - 0s 7ms/step - loss: 4407.8594 - mean_absolute_percentage_error: 23.7360 - mean_absolute_error: 52.2090 - val_loss: 4189.7534 - val_mean_absolute_percentage_error: 23.7800 - val_mean_absolute_error: 50.7819
Epoch 66/120
52/52 [==============================] - 0s 8ms/step - loss: 4895.9082 - mean_absolute_percentage_error: 25.0535 - mean_absolute_error: 54.2685 - val_loss: 4190.3447 - val_mean_absolute_percentage_error: 23.7704 - val_me

In [62]:
#Y_total_train
runs_model.evaluate([team_vector_test,opponent_vector_test,location_vector_test], Y_score_test)

6/6 [==============================] - 0s 7ms/step - loss: 4062.1550 - mean_absolute_percentage_error: 23.7581 - mean_absolute_error: 49.0957


[4062.155029296875, 23.758087158203125, 49.09572982788086]

In [63]:
#runs_model.predict([team_vector_test,opponent_vector_test,location_vector_test])

In [64]:
location_vec_map={}
one_hot_list =[]
locations=[]
for location in location_list:
    location_enc = np.array(enc_map[location]).reshape(1,-1)
    location_vec = location_model.predict(location_enc)
    location_vec_map[location]=location_vec[0]
    one_hot_list.append(np.array(enc_map[location]))
    locations.append(location)
    
location_matrix = location_model.predict(np.stack(one_hot_list))

In [65]:
location_list

['Leeds',
 'Bulawayo Athletic Club',
 'Abu Dhabi',
 'Sharjah Cricket Stadium',
 'Sharjah',
 'Dhaka',
 'Paarl',
 'Melbourne Cricket Ground',
 'Lincoln',
 'Bulawayo',
 'Delhi',
 'Mumbai',
 'Dubai International Cricket Stadium',
 'Nagpur',
 'Kolkata',
 'Melbourne',
 'Potchefstroom',
 'Harare Sports Club',
 'Colombo',
 'Guwahati',
 'Bridgetown',
 'Dublin',
 'Guyana',
 'Chennai',
 'Dunedin',
 'Sydney',
 'Harare',
 'Sydney Cricket Ground',
 'Pallekele International Cricket Stadium',
 'East London',
 'Rajkot',
 'Perth',
 'Bloemfontein',
 'Hyderabad',
 'Whangarei',
 'Taunton',
 'Kanpur',
 'St Lucia',
 'Kandy',
 'Aberdeen',
 'Kirtipur',
 'Hobart',
 'Indore',
 'Nelson',
 'Christchurch',
 'Fatullah',
 'Chattogram',
 'Port Elizabeth',
 'Chittagong',
 'Cape Town',
 'Napier',
 'Chester-le-Street',
 'Birmingham',
 'Belfast',
 'Deventer',
 'Galle International Stadium',
 'Brisbane',
 'London',
 'Mount Maunganui',
 'Pune',
 'Kuala Lumpur',
 'Johannesburg',
 'Lauderhill',
 'Bready',
 'Benoni',
 'Kochi',

In [66]:
len(location_list),len(country_list)

(114, 23)

In [67]:
# location_vec_map={}
# locations=[]
# one_hot_list =[]
# for location in loc_enc_map.keys():
#     location_enc = np.array(loc_enc_map[location]).reshape(1,-1)
#     location_vec = location_model.predict(location_enc)
#     location_vec_map[location]=location_vec[0]
#     one_hot_list.append(np.array(loc_enc_map[location]))
#     locations.append(location)
#     #break
    
# location_matrix = location_model.predict(np.stack(one_hot_list))

In [68]:
#locations = list(location_vec_map.keys())
# location_sim_matrix = np.zeros((len(locations),len(locations)))
# for i,loc_1 in enumerate(locations):
#     for j,loc_2 in enumerate(locations):
#         if loc_1!=loc_2:
#             location_sim_matrix[i,j]=

In [69]:
location_matrix.shape

(114, 10)

In [70]:
len(locations)

114

In [71]:
location_sim_matrix = cosine_similarity(location_matrix)

In [73]:
# loc_sim_df=pd.DataFrame(location_sim_matrix)
# loc_sim_df.columns = location_list
# loc_sim_df['locations']=location_list
# loc_sim_df

In [74]:
for ind,location in enumerate(location_list):
    print('similarity for location ',location)
    for similarity_index in reversed(list(location_sim_matrix[ind].argsort())):
        print('\t',location_list[similarity_index])
        

similarity for location  Leeds
	 Leeds
	 Dublin
	 Lahore
	 Wellington
	 Greater Noida
	 Delhi
	 Perth Stadium
	 Lauderhill
	 Bready
	 Al Amarat
	 Karachi
	 Windhoek
	 Rawalpindi Cricket Stadium
	 Dehra Dun
	 St George's
	 Adelaide
	 Deventer
	 Aberdeen
	 Kirtipur
	 Lucknow
	 Kandy
	 Taunton
	 Sydney
	 Bridgetown
	 Melbourne
	 Gros Islet
	 Sharjah
	 Colombo
	 Chennai
	 Jamaica
	 Kimberley
	 Ranchi
	 Dubai
	 Canberra
	 Manchester
	 Cardiff
	 Nelson
	 Southampton
	 Bengaluru
	 Sharjah Cricket Stadium
	 Melbourne Cricket Ground
	 Lincoln
	 Mount Maunganui
	 Trinidad
	 Chattogram
	 Perth
	 Guyana
	 Paarl
	 Dharmasala
	 Grenada
	 Hyderabad
	 Pallekele International Cricket Stadium
	 Benoni
	 Chittagong
	 Nottingham
	 Kochi
	 Chandigarh
	 Nagpur
	 Antigua
	 Hambantota
	 St Kitts
	 Sylhet
	 Cuttack
	 Barbados
	 Mumbai
	 Adelaide Oval
	 East London
	 Durban
	 Ahmedabad
	 Bulawayo
	 Kanpur
	 Potchefstroom
	 Johannesburg
	 Fatullah
	 Indore
	 Sydney Cricket Ground
	 Bulawayo Athletic Club
	 Rajko

	 Dhaka
	 Dharamsala
	 Antigua
	 Kanpur
	 Chandigarh
	 Bengaluru
	 Chittagong
	 Sylhet
	 Manchester
	 Wellington
	 Lincoln
	 Edinburgh
	 Chennai
	 Guwahati
	 Potchefstroom
	 Colombo
	 Indore
	 Cardiff
	 Dubai
	 Kochi
	 East London
	 Adelaide Oval
	 Whangarei
	 Kuala Lumpur
	 Melbourne Cricket Ground
	 Sydney Cricket Ground
	 Harare
	 Mumbai
	 Benoni
	 Guyana
	 Kimberley
	 Jamaica
	 Cape Town
	 Perth
	 Sharjah Cricket Stadium
	 Barbados
	 Canberra
	 Galle International Stadium
	 Chattogram
	 Harare Sports Club
	 Dubai International Cricket Stadium
	 St Kitts
	 Ahmedabad
	 Johannesburg
	 Kolkata
	 Bulawayo Athletic Club
	 Port Elizabeth
	 Bloemfontein
	 London
	 Bristol
	 Brisbane
	 Mirpur
	 Mount Maunganui
	 Bulawayo
	 Hamilton
	 Pune
	 Chester-le-Street
	 Auckland
	 Visakhapatnam
	 Centurion
	 Dunedin
	 Christchurch
	 Abu Dhabi
	 Rangiri Dambulla International Stadium
	 Thiruvananthapuram
	 Belfast
similarity for location  Melbourne Cricket Ground
	 Melbourne Cricket Ground
	 Galle Int

	 Port Elizabeth
	 Wellington
	 Leeds
	 Jamaica
	 Dubai International Cricket Stadium
	 Sharjah Cricket Stadium
	 Pune
	 Barbados
	 Brisbane
	 Auckland
	 Kanpur
	 Kimberley
	 Manchester
	 Edinburgh
	 St Kitts
	 Dharamsala
	 Queenstown Events Centre
	 Hamilton
	 Hong Kong
	 Bristol
	 Sydney Cricket Ground
	 Johannesburg
	 Melbourne Cricket Ground
	 Bulawayo Athletic Club
	 Paarl
	 Napier
	 Antigua
	 Harare Sports Club
	 Abu Dhabi
	 Guyana
	 East London
	 Nagpur
	 Fatullah
	 St Lucia
	 Mount Maunganui
	 Kuala Lumpur
	 Perth
	 Christchurch
	 Dharmasala
	 Dubai
	 Kolkata
	 Bloemfontein
	 Dunedin
	 Chattogram
	 Chester-le-Street
	 Birmingham
	 Bulawayo
	 Galle International Stadium
	 Visakhapatnam
	 Mirpur
	 London
	 Centurion
	 Belfast
	 Rangiri Dambulla International Stadium
	 Thiruvananthapuram
similarity for location  Potchefstroom
	 Potchefstroom
	 Dhaka
	 Adelaide
	 Kandy
	 Lucknow
	 St George's
	 Dehra Dun
	 Rawalpindi Cricket Stadium
	 Bridgetown
	 Windhoek
	 Taunton
	 Karachi
	 Mel

	 Delhi
	 Potchefstroom
	 Nelson
	 Brisbane
	 Dharmasala
	 Cardiff
	 Leeds
	 Lucknow
	 Sydney
	 Bready
	 Deventer
	 Al Amarat
	 Karachi
	 Kirtipur
	 Aberdeen
	 Rawalpindi Cricket Stadium
	 Dehra Dun
	 St George's
	 Kandy
	 Taunton
	 Adelaide
	 Lauderhill
	 Sharjah
	 Bridgetown
	 Gros Islet
	 Windhoek
	 Melbourne
	 Lincoln
	 Sylhet
	 Lahore
	 Canberra
	 Bulawayo
	 Chennai
	 Kanpur
	 Benoni
	 Dublin
	 Chattogram
	 Nagpur
	 Jamaica
	 Harare
	 Trinidad
	 Dubai
	 Kochi
	 Mumbai
	 Greater Noida
	 Chandigarh
	 Mount Maunganui
	 Cape Town
	 Edinburgh
	 Ranchi
	 Colombo
	 Sharjah Cricket Stadium
	 Abu Dhabi
	 Melbourne Cricket Ground
	 St Kitts
	 Bengaluru
	 Ahmedabad
	 Galle International Stadium
	 Kolkata
	 Sydney Cricket Ground
	 Hyderabad
	 Bristol
	 Adelaide Oval
	 Dhaka
	 Nottingham
	 Southampton
	 Belfast
	 Perth
	 Antigua
	 Port Elizabeth
	 Paarl
	 Barbados
	 Hamilton
	 Grenada
	 Rajkot
	 Manchester
	 Pallekele International Cricket Stadium
	 Durban
	 Chittagong
	 Indore
	 Harare Sports

	 Bready
	 Lauderhill
	 Kandy
	 Lucknow
	 Melbourne
	 Sharjah
	 Windhoek
	 Gros Islet
	 Aberdeen
	 Nelson
	 Johannesburg
	 Adelaide Oval
	 Durban
	 Dublin
	 Lincoln
	 Chittagong
	 St Lucia
	 Chennai
	 Sylhet
	 Hyderabad
	 Lahore
	 Hong Kong
	 Benoni
	 Bristol
	 Perth Stadium
	 Grenada
	 Delhi
	 Mirpur
	 Pallekele International Cricket Stadium
	 Hobart
	 Napier
	 Mumbai
	 Nottingham
	 Chandigarh
	 Guwahati
	 Colombo
	 Whangarei
	 Harare
	 Harare Sports Club
	 Cape Town
	 Kolkata
	 Greater Noida
	 Paarl
	 Trinidad
	 Brisbane
	 Chester-le-Street
	 Bloemfontein
	 Dubai International Cricket Stadium
	 Dunedin
	 Nagpur
	 Dharamsala
	 Mount Maunganui
	 Dhaka
	 Cuttack
	 Manchester
	 Kuala Lumpur
	 Wellington
	 Melbourne Cricket Ground
	 Potchefstroom
	 Pune
	 Christchurch
	 Antigua
	 Auckland
	 Port Elizabeth
	 Hamilton
	 Kimberley
	 Leeds
	 Kanpur
	 Sydney Cricket Ground
	 Ahmedabad
	 Canberra
	 Birmingham
	 Barbados
	 Sharjah Cricket Stadium
	 Queenstown Events Centre
	 Abu Dhabi
	 Cardiff


...

	 Potchefstroom
	 Dhaka
	 Whangarei
	 Cardiff
	 Ahmedabad
	 Guwahati
	 Canberra
	 Rajkot
	 Adelaide Oval
	 Hambantota
	 Cuttack
	 Hobart
	 Port Elizabeth
	 Wellington
	 Leeds
	 Jamaica
	 Dubai International Cricket Stadium
	 Sharjah Cricket Stadium
	 Pune
	 Barbados
	 Brisbane
	 Auckland
	 Kanpur
	 Kimberley
	 Manchester
	 Edinburgh
	 St Kitts
	 Dharamsala
	 Queenstown Events Centre
	 Hamilton
	 Hong Kong
	 Bristol
	 Sydney Cricket Ground
	 Johannesburg
	 Melbourne Cricket Ground
	 Bulawayo Athletic Club
	 Paarl
	 Napier
	 Antigua
	 Harare Sports Club
	 Abu Dhabi
	 Guyana
	 East London
	 Nagpur
	 Fatullah
	 St Lucia
	 Mount Maunganui
	 Kuala Lumpur
	 Perth
	 Christchurch
	 Dharmasala
	 Dubai
	 Kolkata
	 Bloemfontein
	 Dunedin
	 Chattogram
	 Chester-le-Street
	 Birmingham
	 Bulawayo
	 Galle International Stadium
	 Visakhapatnam
	 Mirpur
	 London
	 Centurion
	 Belfast
	 Rangiri Dambulla International Stadium
	 Thiruvananthapuram
similarity for location  Kimberley
	 Kimberley
	 Lahore
	 D

	 Indore
	 Antigua
	 Nagpur
	 Ahmedabad
	 Edinburgh
	 Lincoln
	 Perth
	 Leeds
	 Sharjah Cricket Stadium
	 Port Elizabeth
	 Dubai
	 East London
	 Bulawayo Athletic Club
	 Cardiff
	 Pune
	 Bloemfontein
	 Harare Sports Club
	 Benoni
	 Canberra
	 Dubai International Cricket Stadium
	 Adelaide Oval
	 Visakhapatnam
	 Dunedin
	 Kimberley
	 Hamilton
	 Christchurch
	 Brisbane
	 Melbourne Cricket Ground
	 Johannesburg
	 Bristol
	 London
	 Chattogram
	 Guyana
	 Centurion
	 Abu Dhabi
	 Kolkata
	 Galle International Stadium
	 Mount Maunganui
	 Chester-le-Street
	 Mirpur
	 Bulawayo
	 Rangiri Dambulla International Stadium
	 Thiruvananthapuram
	 Belfast
similarity for location  St Kitts
	 St Kitts
	 Barbados
	 Jamaica
	 Colombo
	 Mumbai
	 Cuttack
	 Cape Town
	 Chennai
	 Kochi
	 Perth
	 Dehra Dun
	 Rawalpindi Cricket Stadium
	 Lucknow
	 Sharjah
	 Adelaide
	 Taunton
	 Kandy
	 Deventer
	 Sydney
	 St George's
	 Karachi
	 Bridgetown
	 Windhoek
	 Kirtipur
	 Melbourne
	 Al Amarat
	 Bready
	 Lauderhill
	 Abe

	 Hambantota
	 Hong Kong
	 Cuttack
	 Nottingham
	 Chittagong
	 Southampton
	 Trinidad
	 Rajkot
	 Dublin
	 Hyderabad
	 Chennai
	 Bengaluru
	 Durban
	 Gros Islet
	 Melbourne
	 Lucknow
	 Bridgetown
	 Adelaide
	 St George's
	 Dehra Dun
	 Rawalpindi Cricket Stadium
	 Kirtipur
	 Taunton
	 Karachi
	 Aberdeen
	 Kandy
	 Al Amarat
	 Windhoek
	 Deventer
	 Bready
	 Lauderhill
	 Sharjah
	 Sydney
	 Ranchi
	 Adelaide Oval
	 Indore
	 Guwahati
	 Pallekele International Cricket Stadium
	 Mumbai
	 Colombo
	 Kochi
	 Kuala Lumpur
	 Napier
	 St Lucia
	 Bristol
	 Hobart
	 Nelson
	 Chandigarh
	 Paarl
	 Dubai International Cricket Stadium
	 Whangarei
	 Johannesburg
	 Lincoln
	 Grenada
	 Queenstown Events Centre
	 Harare
	 Barbados
	 Greater Noida
	 Sylhet
	 Lahore
	 Delhi
	 Wellington
	 Bulawayo Athletic Club
	 Fatullah
	 Dharamsala
	 Perth Stadium
	 Perth
	 Cape Town
	 Sharjah Cricket Stadium
	 Leeds
	 Cardiff
	 Canberra
	 Dhaka
	 Manchester
	 Brisbane
	 St Kitts
	 Dunedin
	 Jamaica
	 Harare Sports Club
	 Pun

	 Grenada
	 St Lucia
	 Dubai
	 Ahmedabad
	 Napier
	 Bloemfontein
	 Antigua
	 Galle International Stadium
	 Paarl
	 Queenstown Events Centre
	 Fatullah
	 East London
	 Rangiri Dambulla International Stadium
	 Edinburgh
	 Birmingham
	 Centurion
	 London
	 Visakhapatnam
	 Thiruvananthapuram
similarity for location  Antigua
	 Antigua
	 Dharamsala
	 Sydney Cricket Ground
	 Melbourne Cricket Ground
	 Manchester
	 East London
	 Ranchi
	 Pallekele International Cricket Stadium
	 Dublin
	 Lahore
	 Galle International Stadium
	 Southampton
	 Chittagong
	 Hyderabad
	 Nelson
	 Lincoln
	 Chandigarh
	 Durban
	 Bready
	 Kandy
	 Lucknow
	 St George's
	 Taunton
	 Dehra Dun
	 Adelaide
	 Rawalpindi Cricket Stadium
	 Kirtipur
	 Windhoek
	 Gros Islet
	 Karachi
	 Deventer
	 Al Amarat
	 Lauderhill
	 Aberdeen
	 Melbourne
	 Sharjah
	 Sydney
	 Bridgetown
	 Grenada
	 Perth Stadium
	 Greater Noida
	 Harare Sports Club
	 Bengaluru
	 Potchefstroom
	 Nottingham
	 Colombo
	 Benoni
	 Kanpur
	 Napier
	 Hamilton
	 Leeds

	 Kanpur
	 Bloemfontein
	 Nagpur
	 Port Elizabeth
	 Chester-le-Street
	 Dubai
	 Dharmasala
	 Fatullah
	 Jamaica
	 Dunedin
	 Sharjah Cricket Stadium
	 Guyana
	 Bulawayo
	 Birmingham
	 Kolkata
	 Abu Dhabi
	 Bulawayo Athletic Club
	 St Kitts
	 Perth
	 London
	 Mirpur
	 Chattogram
	 Visakhapatnam
	 Centurion
	 Rangiri Dambulla International Stadium
	 Belfast
	 Thiruvananthapuram


In [75]:
# first_innings_vec_map={}
# teams=[]
# batting_one_hot_list =[]
# for team in country_enc_map.keys():
#     country_enc = np.array(country_enc_map[team]).reshape(1,-1)
#     country_vec = team_model.predict(country_enc)
#     first_innings_vec_map[team]=country_vec[0]
#     batting_one_hot_list.append(np.array(country_enc_map[team]))
#     teams.append(team)
#     #break
    
# team_matrix = team_model.predict(np.stack(batting_one_hot_list))

In [76]:
# team_sim_matrix = cosine_similarity(team_matrix)

In [77]:
# for ind,team in enumerate(teams):
#     print('similarity for location ',team)
#     for similarity_index in reversed(list(team_sim_matrix[ind].argsort())):
#         print('\t',teams[similarity_index])

In [78]:
#team_model.predict(np.array(country_enc_map['India']).reshape(1,-1))

In [79]:
#opponent_model.predict(np.array(country_enc_map['India']).reshape(1,-1))

In [80]:
# second_innings_vec_map={}
# #teams=[]
# #second_one_hot_list =[]
# for team in country_enc_map.keys():
#     country_enc = np.array(country_enc_map[team]).reshape(1,-1)
#     country_vec = opponent_model.predict(country_enc)
#     second_innings_vec_map[team]=country_vec[0]
#     #second_one_hot_list.append(np.array(country_enc_map[team]))
#     #teams.append(team)
#     #break
    
# #team_matrix = team_model.predict(np.stack(batting_one_hot_list))

In [81]:
import pickle
# pickle.dump(location_vec_map,open('location_vector_map.pkl','wb'))
# pickle.dump(first_innings_vec_map,open('first_innings_vec_map.pkl','wb'))
# pickle.dump(second_innings_vec_map,open('second_innings_vec_map.pkl','wb'))

# pickle.dump(enc_map,open('country_location_enc_map.pkl','wb'))
# pickle.dump(decode_map,open('country_location_dec_map.pkl','wb'))




In [82]:
def store_model(model,name):
    # serialize model to JSON
    model_json = model.to_json()
    with open(name+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name+".h5")
    print("Saved model to disk")

In [83]:
# store_model(runs_model,'run_model_embedding')
# store_model(team_model,'first_inn_model_embedding')
# store_model(opponent_model,'second_inn_model_embedding')
# store_model(group_encode_model,'group_encode_model')

Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk


In [85]:
# pickle.dump(country_enc_map,open('team_oh_map.pkl','wb'))
# pickle.dump(loc_enc_map,open('loc_oh_map.pkl','wb'))
